In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import numpy.random as rn
import networkx as nx


In [3]:
A = np.array([[0, 1, 0, 0, 0, 0],[1, 0, 1, 0, 1, 0],[0, 1, 0, 1, 0, 0],[0, 0, 1, 0, 1, 0],[0, 1, 0, 1, 0, 1],[0, 0, 0, 0, 1, 0]])
print(A)

[[0 1 0 0 0 0]
 [1 0 1 0 1 0]
 [0 1 0 1 0 0]
 [0 0 1 0 1 0]
 [0 1 0 1 0 1]
 [0 0 0 0 1 0]]


In [21]:
energies = rn.exponential(1, size=(6))
print(energies)

[0.08355668 0.68561675 0.90880733 0.53774405 0.31794683 0.37527759]


In [22]:
peq = np.exp(-energies)
Z = np.sum(peq)
peq = peq/Z

In [24]:
N = len(peq)
Pmatrix = np.zeros((N, N))
for i in range(N):
    Pmatrix[i, i] = np.sqrt(peq[i])

print(Pmatrix)

[[0.490361   0.         0.         0.         0.         0.        ]
 [0.         0.36289437 0.         0.         0.         0.        ]
 [0.         0.         0.32457496 0.         0.         0.        ]
 [0.         0.         0.         0.39074226 0.         0.        ]
 [0.         0.         0.         0.         0.43613279 0.        ]
 [0.         0.         0.         0.         0.         0.42380837]]


In [35]:
L = np.zeros((N, N))

for i in range(N):
    for j in range(N):
        if A[i, j] == 1:
            L[i, j] = np.exp(energies[j])
    L[i, i] = - np.sum(L[i, :])

Ls = np.matmul(np.linalg.inv(Pmatrix), np.matmul(L, Pmatrix))
print(Ls)

ew, ev = np.linalg.eigh(Ls)
print(ew)

D = np.matmul(ev.T, np.matmul(Ls, ev))
print(D)



[[-1.9849957   1.46900707  0.          0.          0.          0.        ]
 [ 1.46900707 -4.94281135  2.21934485  0.          1.65166156  0.        ]
 [ 0.          2.21934485 -3.6971357   2.06117394  0.          0.        ]
 [ 0.          0.          2.06117394 -3.85566452  1.53394898  0.        ]
 [ 0.          1.65166156  0.          1.53394898 -5.15253107  1.41426818]
 [ 0.          0.          0.          0.          1.41426818 -1.37430319]]
[-8.40733588e+00 -5.08212840e+00 -4.59498810e+00 -1.83079608e+00
 -1.09219305e+00 -1.45627060e-16]
[[-8.40733588e+00  8.32667268e-17  9.08995101e-16 -9.57567359e-16
   9.71445147e-17  1.83066610e-16]
 [ 6.38378239e-16 -5.08212840e+00 -1.47104551e-15 -3.88578059e-16
  -8.32667268e-17 -2.54423408e-17]
 [ 1.33226763e-15 -1.58206781e-15 -4.59498810e+00  1.42247325e-15
  -7.35522754e-16 -4.88478953e-17]
 [-2.22044605e-16 -3.33066907e-16  1.30451205e-15 -1.83079608e+00
  -2.22044605e-16 -3.71353781e-17]
 [-3.33066907e-16  2.22044605e-16 -9.99200722e